**Library Imports**

In [ ]:
!pip install imblearn

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import math
import nltk
from nltk.corpus import words
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.metrics import accuracy_score
from imblearn.ensemble import BalancedBaggingClassifier
from imblearn.under_sampling import RandomUnderSampler
from imblearn.ensemble import EasyEnsembleClassifier
from sklearn.svm import LinearSVC

Mount Google Drive and Load Dataset

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')
# Load data
df = pd.read_csv('/content/drive/MyDrive/password_strength_dataset/cleaned_password_strength.csv')

Mounted at /content/drive


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18192 entries, 0 to 18191
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   strength               18192 non-null  float64
 1   length                 18192 non-null  int64  
 2   digit_count            18192 non-null  int64  
 3   lowercase_count        18192 non-null  int64  
 4   uppercase_count        18192 non-null  int64  
 5   special_count          18192 non-null  int64  
 6   entropy                18192 non-null  float64
 7   sequential_chars       18192 non-null  int64  
 8   dictionary_word_count  18192 non-null  int64  
dtypes: float64(2), int64(7)
memory usage: 1.2 MB


In [ ]:
df.describe()

,strength,length,digit_count,lowercase_count,uppercase_count,special_count,entropy,sequential_chars,dictionary_word_count
count,18192.000000,18192.00000,18192.000000,18192.000000,18192.000000,18192.000000,18192.000000,18192.000000,18192.000000
mean,1.610378,14.87555,4.877144,6.943382,2.112082,0.942502,3.205205,4.640996,0.167051
std,0.551277,5.18104,3.293584,5.153441,3.120194,1.477701,0.639367,1.745288,0.404018
min,0.000000,1.00000,0.000000,0.000000,0.000000,0.000000,-0.000000,0.000000,0.000000
25%,1.000000,12.00000,2.000000,3.000000,0.000000,0.000000,2.855389,4.000000,0.000000
50%,2.000000,15.00000,4.000000,6.000000,1.000000,0.000000,3.334679,5.000000,0.000000
75%,2.000000,17.00000,7.000000,10.000000,3.000000,1.000000,3.640224,5.000000,0.000000
max,2.000000,220.00000,39.000000,164.000000,80.000000,21.000000,5.684569,73.000000,8.000000


In [ ]:
df.head()

,strength,length,digit_count,lowercase_count,uppercase_count,special_count,entropy,sequential_chars,dictionary_word_count
0,1.0,8,4,4,0,0,2.500000,2,0
1,1.0,8,4,4,0,0,2.500000,2,1
2,1.0,9,2,7,0,0,2.947703,3,0
3,1.0,8,3,5,0,0,3.000000,2,0
4,1.0,11,1,10,0,0,3.459432,3,0


# Select 50% of the entire data for the experiment


In [ ]:
df = df.sample(frac=0.5, random_state=42)

Split Data for Model Training

In [ ]:
X = df.drop(columns=['strength'])  # Features
y = df['strength']                 # Target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)


Feature Scaling

In [ ]:
# Scale both train and test
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert back to DataFrames with column names
X_train = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_test = pd.DataFrame(X_test_scaled, columns=X_test.columns)

In [ ]:
# Apply random undersampling to the training set
#undersampler = RandomUnderSampler(random_state=42)
#X_train_resampled, y_train_resampled = undersampler.fit_resample(X_train, y_train)

# Training and Evaluation

# RF

In [ ]:

rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [ ]:


# Train on scaled data
classifier = RandomForestClassifier(n_estimators=100, random_state=42)
classifier.fit(X_train, y_train)

# Predict
y_pred = classifier.predict(X_test)       # Show confusion matrix

In [ ]:
print(accuracy_score(y_test, y_pred))              # Accuracy score


0.9985342616342983


In [ ]:
print(classification_report(y_test, y_pred, digits=4))  # Precision, recall, f1-score

              precision    recall  f1-score   support

         0.0     0.9902    0.9712    0.9806       104
         1.0     0.9964    0.9988    0.9976       835
         2.0     1.0000    1.0000    1.0000      1790

    accuracy                         0.9985      2729
   macro avg     0.9955    0.9900    0.9927      2729
weighted avg     0.9985    0.9985    0.9985      2729



In [ ]:

error_rate = 1 - accuracy_score(y_test, y_pred)
print(f"Error Rate: {error_rate:.4f}")


Error Rate: 0.0015


In [ ]:
# Evaluation
print(confusion_matrix(y_test, y_pred))

[[ 101    3    0]
 [   1  834    0]
 [   0    0 1790]]


# SVM

In [ ]:


# Train on scaled data
svc = make_pipeline(StandardScaler(), LinearSVC(random_state=42))
svc.fit(X_train, y_train)

# Predict
y_pred = svc.predict(X_test)       # Show confusion matrix

In [ ]:
print(accuracy_score(y_test, y_pred))              # Accuracy score


0.9805789666544522


In [ ]:
print(classification_report(y_test, y_pred, digits=4))  # Precision, recall, f1-score

              precision    recall  f1-score   support

         0.0     1.0000    0.4904    0.6581       104
         1.0     0.9403    1.0000    0.9692       835
         2.0     1.0000    1.0000    1.0000      1790

    accuracy                         0.9806      2729
   macro avg     0.9801    0.8301    0.8758      2729
weighted avg     0.9817    0.9806    0.9776      2729



In [ ]:

error_rate = 1 - accuracy_score(y_test, y_pred)
print(f"Error Rate: {error_rate:.4f}")


Error Rate: 0.0194


In [ ]:
# Evaluation
print(confusion_matrix(y_test, y_pred))

[[  51   53    0]
 [   0  835    0]
 [   0    0 1790]]


# Ensemble Voting

In [ ]:
# Define individual models (ensure SVC is in a pipeline to handle scaling)
svc = make_pipeline(StandardScaler(), LinearSVC(random_state=42))
rf = RandomForestClassifier(n_estimators=100, random_state=42)

# Create Voting Ensemble (Hard Voting)
ensemble = VotingClassifier(estimators=[
    ('svc', svc),
    ('rf', rf)
], voting='hard')
ensemble.fit(X_train, y_train)
# Predict
y_pred = ensemble.predict(X_test)

In [ ]:
print(accuracy_score(y_test, y_pred))              # Accuracy score


0.9989006962257237


In [ ]:
print(classification_report(y_test, y_pred, digits=4))  # Precision, recall, f1-score

              precision    recall  f1-score   support

         0.0     0.9903    0.9808    0.9855       104
         1.0     0.9976    0.9988    0.9982       835
         2.0     1.0000    1.0000    1.0000      1790

    accuracy                         0.9989      2729
   macro avg     0.9960    0.9932    0.9946      2729
weighted avg     0.9989    0.9989    0.9989      2729



In [ ]:

error_rate = 1 - accuracy_score(y_test, y_pred)
print(f"Error Rate: {error_rate:.4f}")


Error Rate: 0.0011


In [ ]:
# Evaluation
print(confusion_matrix(y_test, y_pred))

[[ 102    2    0]
 [   1  834    0]
 [   0    0 1790]]


# Bagging

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from imblearn.ensemble import BalancedBaggingClassifier

# Corrected: use 'estimator' instead of 'base_estimator'
boost_bagging = BalancedBaggingClassifier(estimator=DecisionTreeClassifier(), random_state=42)
boost_bagging.fit(X_train, y_train)


BalancedBaggingClassifier(estimator=DecisionTreeClassifier(), random_state=42)

In [ ]:
y_pred = boost_bagging.predict(X_test)        # Predict on test set

# Evaluation
print(confusion_matrix(y_test, y_pred))            # Show confusion matrix

[[ 104    0    0]
 [   7  828    0]
 [   0    0 1790]]


In [ ]:
print(accuracy_score(y_test, y_pred))              # Accuracy score


0.997434957860022


In [ ]:
print(classification_report(y_test, y_pred, digits=4))  # Precision, recall, f1-score

              precision    recall  f1-score   support

         0.0     0.9369    1.0000    0.9674       104
         1.0     1.0000    0.9916    0.9958       835
         2.0     1.0000    1.0000    1.0000      1790

    accuracy                         0.9974      2729
   macro avg     0.9790    0.9972    0.9877      2729
weighted avg     0.9976    0.9974    0.9975      2729



In [ ]:

error_rate = 1 - accuracy_score(y_test, y_pred)
print(f"Error Rate: {error_rate:.4f}")


Error Rate: 0.0026


# Boosting

In [ ]:
# Training the Random Forest Classifier
boost = EasyEnsembleClassifier()
boost.fit(X_train, y_train)

EasyEnsembleClassifier()

In [ ]:
y_pred = boost.predict(X_test)        # Predict on test set

# Evaluation
print(confusion_matrix(y_test, y_pred))            # Show confusion matrix

[[ 104    0    0]
 [   8  827    0]
 [   0    0 1790]]


In [ ]:
print(accuracy_score(y_test, y_pred))              # Accuracy score


0.9970685232685965


In [ ]:
print(classification_report(y_test, y_pred, digits=4))  # Precision, recall, f1-score

              precision    recall  f1-score   support

         0.0     0.9286    1.0000    0.9630       104
         1.0     1.0000    0.9904    0.9952       835
         2.0     1.0000    1.0000    1.0000      1790

    accuracy                         0.9971      2729
   macro avg     0.9762    0.9968    0.9860      2729
weighted avg     0.9973    0.9971    0.9971      2729



In [ ]:

error_rate = 1 - accuracy_score(y_test, y_pred)
print(f"Error Rate: {error_rate:.4f}")


Error Rate: 0.0029


In [ ]:
# Evaluation
print(confusion_matrix(y_test, y_pred))

[[ 104    0    0]
 [   8  827    0]
 [   0    0 1790]]


In [ ]:
# Define the new confusion matrices for each model using 50% of data
confusion_matrices_50 = {
    "Random Forest": [[101, 3, 0], [1, 834, 0], [0, 0, 1790]],
    "SVM": [[51, 53, 0], [0, 835, 0], [0, 0, 1790]],
    "Voting": [[102, 2, 0], [1, 834, 0], [0, 0, 1790]],
    "Bagging": [[104, 0, 0], [7, 828, 0], [0, 0, 1790]],
    "Boosting": [[104, 0, 0], [8, 827, 0], [0, 0, 1790]]
}

# Initialize a list to store the metrics
metrics_50 = []

# Calculate metrics for each model
for model, matrix in confusion_matrices_50.items():
    TP = matrix[0][0] + matrix[1][1] + matrix[2][2]
    FP = sum([matrix[i][j] for i in range(3) for j in range(3) if i != j])
    FN = FP  # Same as FP in this multiclass summing method
    total = sum(sum(row) for row in matrix)
    accuracy = TP / total

    metrics_50.append({
        "Model": model,
        "True-Pos": f"{(TP / total * 100):.1f}%",
        "True-Neg": "N/A",
        "False-Pos": f"{(FP / total * 100):.1f}%",
        "False-Neg": f"{(FN / total * 100):.1f}%"
    })

# Create DataFrame
metrics_df_50 = pd.DataFrame(metrics_50)

# Show the table to the user
tools.display_dataframe_to_user(name="Table 8: Model Confusion Matrix Metrics (50% Data)", dataframe=metrics_df_50)
